1. Datenaufbereitung: Laden und inspizieren Sie die Daten.
2. Stationarität prüfen: Überprüfen Sie, ob die Zeitreihe stationär ist.
3. Differenzierung: Falls die Zeitreihe nicht stationär ist, machen Sie sie stationär.
4. Modell identifizieren: Bestimmen Sie die Parameter p, d und q für das ARIMA-Modell.
5. Modell schätzen: Schätzen Sie die Modellparameter.
6. Modell diagnostizieren: Überprüfen Sie die Modellanpassung.
7. Prognosen erstellen: Verwenden Sie das Modell zur Vorhersage.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import scipy.stats as stats